<a href="https://colab.research.google.com/github/MatzeLopi/KIT-2400024/blob/main/DLNN_SS24_Praktikum1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum 1 - Simple Neural Network with Numpy and Pytorch

Note: the praktikums are for your own practice. They will **not be graded**!

You have around one week to work on it. Then we will go over the solutions together in the praktikum time slots!

Remember to make a copy of this notebook to your own Colab. Changes made directly here will not be stored!

In this praktikum, we will build a perceptron model and a Multi-Layer Perceptron (MLP) model **from scratch** with numpy. This is intended to help you understand in details the internal working of neural networks.

Next, we will proceed with building an MLP with [**Pytorch**](https://pytorch.org/get-started/locally/), which is a widely-used framework for building deep learning models.

Our first challenge is solving the **XOR task** that you've seen in the lecture, before we move to a slightly more complex problem, namely the **Iris dataset**.

**Notice**: Whenenver you see an ellipsis `...` or TODO comment, you're supposed to insert code or text answers.

## Excercise 1: XOR Task with Single Perceptron (from scratch)

XOR (exclusive OR) is a logic function that gives 1 as an output when the number of true inputs is odd, otherwise it outputs a 0. Our goal is to model this function using neurons. We'll start with a single neuron.

<center><img src="https://www.xplore-dna.net/pluginfile.php/286/mod_page/content/21/Tabelle%20-%20XOR.png" width="250"/></center>

Let's start with importing some necessary dependencies that we will need throughout the notebook.

In [129]:
import numpy as np

In the first part of this exercise you'll build a perceptron, a single neuron, that takes two binary input values and returns a binary output value.

<center><img src="https://i.stack.imgur.com/eBSki.jpg" width="280" />

<center><img src="" width="280"/>

Perceptron can be seen as a single neuron, mapping an input $\textbf{x}$ to an output $o$ using weights $\textbf{w}$ and a bias $b$. $\cdot$ is the dot product.

$o = \textbf{w}\cdot \textbf{x}+b$

#### Perceptron Update Rule


Perceptron Update Rule is a process that is specific to the training of a single-perceptron model, which we can apply to binary classification problems. This process [has been proven to converge](https://www.cs.columbia.edu/~mcollins/courses/6998-2012/notes/perc.converge.pdf) if the data is linearly seperatable and the learning rate is small enough.


Let's use it here to have a first baseline.

For classification problems $0>o$ is interpreted as class 1, and $o<0$ is interpreted as class 0.

For updating the associated weights, we can use the following update rule:

$w_i = w_i + \nabla w_i$

where

$\nabla w_i = \eta(t-o)x_i$

- $t$ is the target
- $o$ is the output
- $\eta$ is the learning rate (a small constant)

### Implementation of a Perceptron

In [130]:
from typing import Callable

def step_function(x) -> int:
    """ Step function that returns 1 if x >= 0, 0 otherwise"""
    return 1 if x >= 0 else 0

class Perceptron():
    """ Implementation of a Perceptron following the above instructions

    Attributes:
        neuron_weights (np.array): Weights of the perceptron
        bias (int): Bias of the perceptron
    """
    def __init__(self):
        """ Initialize the perceptron weights and bias"""
        # TODO:
        # Initialize weights
        # For perceptrons, it's possible to initialize all weights with 0

        self.neuron_weights = np.zeros(2)
        self.bias = 0

    def forward_pass(self, x:np.array,activation_func: Callable = step_function) -> np.array:
        """Forward pass of the perceptron

        Args:
            x (np.array): Input from the previous layer / input data

        Returns:
            np.array: Weighted sum of the input data + bias
        """
        output = sum(self.neuron_weights * x) + self.bias

        return output if activation_func is None else activation_func(output)

    def perceptron_update_rule(self, target:np.array, prediction:np.array, x:np.array, learning_rate=1) -> None:
        """ Perform update of the perceptron following the rule defined above

        Args:
            target (np.array): Target value
            prediction (np.array): Predicted value
            x (np.array): Input data
            learning_rate (int, optional): Learning rate. Defaults to 1.

        Returns:
            None

        """
        new_weights = self.neuron_weights.copy()
        for index, _ in enumerate(self.neuron_weights):

            weight_delta = learning_rate * ((target - prediction) * x[index])
            new_weights[index] += weight_delta


        self.neuron_weights = new_weights

    def train(self, input_data:np.ndarray, targets:np.ndarray):
        """ Training loop for the perceptron
        
        Args:
            input_data: Multi-dimensional array that contains all inputs
            targets: Array that contains all targets
        """
        for data, target in zip(input_data, targets):
            prediction = self.forward_pass(data,step_function)
            print(data, prediction, target)
            self.perceptron_update_rule(target, prediction, data)

    def inference(self, input_data:np.ndarray):
        """ Inference of the perceptron

        Args:
            input_data (_type_): _description_

        Returns:
            _type_: _description_
        """
        # TODO
        outputs = []
        for x in input_data:
            outputs.append(self.forward_pass(x,step_function))

        return np.array(outputs)

### Training

In [131]:
perceptron = Perceptron()

# TODO
# Define input data and targets
input_data = [[0,0],[1,0],[0,1],[1,1]]
targets = [0,1,1,0]

perceptron.train(input_data, targets)

[0, 0] 1 0
[1, 0] 1 1
[0, 1] 1 1
[1, 1] 1 0


### Inference

In [132]:
# TODO
# Test the trained model

predictions = perceptron.inference(input_data)
print(predictions)

[1 0 0 0]


### Evaluation

For evaluation, we will need to consider appropriate metrics. For classification tasks, **accuracy** is one of the most common metrics.

It is defined as:

$\textrm{Accuracy}=\frac{1}{N}\sum_i^N1(y_i=\hat{y}_i)$

where $y$ is an array of our target values, and $\hat{y}$ is an array of our predictions.

For accuracy, if outputs are probabilities, there needs to be a threshold for transforming logit predictions to binary `(0,1)` predictions. We will set this threshold to `0.5`. For our perceptron this is not needed, since we already output binary values, however, we will use the `accuracy` function later on, so the predictions should be considered to be probabilities.

In [133]:
def accuracy(predictions: np.ndarray, targets: np.ndarray, threshold=0.5) -> float:
    """ Function to evaluate the accuracy of the model

    Args:
        predictions (np.ndarray): Predictions of the model
        targets (np.ndarray): Actual Targets
        threshold (float, optional): Threshold to convert percentage to binary. Defaults to 0.5.

    Returns:
        float: accuracy of the model
    """

    assert len(predictions) == len(targets)

    accuracy_value = 0
    for prediction, target in zip(predictions, targets):
        if prediction >= threshold:
            prediction = 1
        else:
            prediction = 0
        
        if prediction == target:
            accuracy_value += 1

    return accuracy_value / len(targets)

In [134]:

accuracy_value = accuracy(predictions, targets) 
print(accuracy_value)

0.25


You will see that it is not possible to get to 100% accuracy, since XOR is not a linear-separatable problem.

## Excercise 2: XOR Task with MLP (from scratch)

As mentioned in the lecture, unlike a single perceptron, Multi-Layer Perceptron (MLP) can deal with problems that are non-linearly-separatable like XOR.

Now we will try to implement an MLP with 3 hidden layers and a hidden dimension of 3. We will also add an activiation function to introduce nonlinearity in our hidden layers.

<img src="https://i.imgur.com/IUQ05Ol.png">

### Initializing Weights

Xavier intitialization is commonly used to initialize the weights of a network. It is a random uniform distribution that’s bounded between $\pm\frac{\sqrt{6}}{\sqrt{n_i+n_{i+1}}}$ where $n_i$ is the number of incoming network connections, and $n_{i+1}$ is the number of outgoing network connections.

In [135]:
def xavier_initialization(input_size, output_size) -> np.ndarray:
    """ Returns a numpy array of initialized weights """
    bound = np.sqrt(6) / np.sqrt(input_size + output_size)
    weights = np.random.uniform(-bound, bound, size=(input_size, output_size))
    return weights


### Feed-Forward Layer


A feed-forward layer applies a linear transformation to the input $x$ using a weight matrix $\textbf{W}$ and a bias vector $b$:

$z = x\textbf{W}^T+b$

Derivatives:
$$
\dfrac{dz}{dw_i} = x_i
$$

$$
\dfrac{dz}{db} = 1
$$

$$
\dfrac{dz}{dx_i} = w_i
$$

In [136]:
class FeedForwardLayer():
    """ Implementation of a feedforward layer

    Attributes:
        weights (np.ndarray): Weights of the layer. Matrix of shape (input_size, output_size)
        biases (np.ndarray): Biases of the layer. Vector of shape (1, output_size)
    """
    def __init__(self, input_size:int, output_size:int):
        """
        Args:
            input_size (int): Input shape of the layer
            output_size (int): Output of the  
        """
        # initialize weights with Xavier intitialization and biases with zeros
        self.weights = xavier_initialization(input_size, output_size)
        self.biases = np.zeros(output_size)

    def forward(self, x) -> np.ndarray:
        """
        Forward pass

        Args:
            x (np.ndarray): Input to the layer
        """
        self.x = np.array(x)

        # Calculate the output by applying an activation function
        output = np.dot(self.weights.T, self.x) + self.biases
        print(f"Output of the network, {output}")

        assert len(output) == self.output_size
        print(f"Output of the network")
        return output

    def backward(self, d_values, learning_rate):
        """
        Backpropagation of the layer

        Args:
            d_values (np.ndarray): Derivative of the loss with respect to the output of the layer
            learning_rate (float): Learning rate for gradient descent

        Returns:
            np.ndarray: Derivative of the loss with respect to the input of the layer
        """
        # Calculate the derivative with respect to the weights
        d_weights = np.dot(self.x.T, d_values)

        # Calculate the derivative with respect to the biases
        d_biases = np.sum(d_values, axis=0, keepdims=True)

        # Calculate the derivative with respect to the input
        d_inputs = np.dot(d_values, self.weights.T)

        # Update the weights and biases using the learning rate and their derivatives
        self.weights -= learning_rate * d_weights
        self.biases -= learning_rate * d_biases

        return d_inputs

**Question**: Why do we need to calculate `d_weights`, `d_biases` and `d_inputs`?

**Answer**: To enable the network to learn using back propagation

### Adding Nonlinearity

For nonlinearity, you should implement Rectified Linear Unit (ReLU) and apply it between the hidden layers to provide nonlinearity to the network.

$$ y = max(0, x) $$

When we examine the ReLU behavior, it looks like it is the combination of two different linear functions. This property makes the training easier yet effective since ReLU does not have any learnable parameters as well as easy to apply because of combination of two simple linear functions.



<center><figure><img src="https://machinelearningmastery.com/wp-content/uploads/2018/10/Line-Plot-of-Rectified-Linear-Activation-for-Negative-and-Positive-Inputs.png" width="450"/><figcaption>Graph of the ReLU activation function. <a href="https://machinelearningmastery.com/wp-content/uploads/2018/10/Line-Plot-of-Rectified-Linear-Activation-for-Negative-and-Positive-Inputs.png">Image source</a></figcaption></figure></center>


Derivative of ReLU:

$\dfrac{dy}{dx} = 1 $ if $x >= 0$

$\dfrac{dy}{dx} = 0 $ if $x < 0$

In [137]:
class ReluActivationFunction():
    def forward(self, x):
        self.x = x
        output = max(x)
        return output if output > 0 else 0

    def backward(self, d_values):
        # Calculate the derivative of the ReLU activation function
        derivative = 1 if self.x >= 0 else 0
        d_output = d_values * derivative
        return d_output



### Backpropagation

The perceptron algorithm can't be generalized to MLP, that's why we will now use **backpropagation**.

<center><img src="https://i.imgur.com/LgBzpYD.png" width="400" /></center>

### Loss Function: Binary Cross Entropy

Backpropagation requires us to have a **loss function**.

$$ L = - (y \times ln(o)+(1-y) \times ln(1-o)) $$

[Derivative](https://www.google.com/search?q=cross+entropy+loss+derivative&sca_esv=6915796dc894fc83&sca_upv=1&rlz=1C5CHFA_enVN752VN752&udm=2&biw=1309&bih=708&sxsrf=ACQVn09fs99X4SFZJk0xmct6PWrepRzpxQ%3A1713181875984&ei=sxQdZuGxO8eE9u8P9oiI6AI&ved=0ahUKEwih15zpk8SFAxVHgv0HHXYEAi0Q4dUDCBA&uact=5&oq=cross+entropy+loss+derivative&gs_lp=Egxnd3Mtd2l6LXNlcnAiHWNyb3NzIGVudHJvcHkgbG9zcyBkZXJpdmF0aXZlMgQQIxgnMgUQABiABDIHEAAYgAQYGEjkBVDRA1jRA3ACeACQAQCYATCgATCqAQExuAEDyAEA-AEBmAIBoAIzmAMAiAYBkgcBMaAHsAM&sclient=gws-wiz-serp#vhid=fKdGq3KS8we6mM&vssid=mosaic):

$$
\dfrac{dL}{do} = \dfrac{-y}{o} + \dfrac{1-y}{1-o}
$$

In [138]:
class BinaryCrossEntropy():

    def forward(self, output, target):

        # TODO
        # implement Binary Cross-Entrops loss function for output, target

        loss = -(target * np.log(output) + (1 - target) * np.log(1 - output))

        assert loss > 0

        return loss

    def backward(self, output, target):
        # Calculate the gradient with respect to the output
        return (- target / output) + (1 - target) / (1 - output)

### Sigmoid Activation Function

For a binary classification problem, we can use the sigmoid activation function in the output layer which outputs values in the range of 0 and 1. So, for a positive case (class 1), we can interpret $p_1 = \sigma(o)$ as the probability of that class, while $p_0 = 1 - p_1$ can be seen the probability of the negative case (class 0).

**Sigmoid function**:
$$
\sigma(x) = \dfrac{1}{1 + e^{-x}}
$$

[Derivative](https://hausetutorials.netlify.app/posts/2019-12-01-neural-networks-deriving-the-sigmoid-derivative/#:~:text=The%20derivative%20of%20the%20sigmoid%20function%20%CF%83(x)%20is%20the,1%E2%88%92%CF%83(x).):
$$
\dfrac{d\sigma}{dx} = \sigma(x)(1-\sigma(x))
$$



In [139]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class SigmoidActivationFunction():

    def forward(self, x):
        # TODO
        # implement Sigmoid function for the input_data
        self.x = x

        output = sigmoid(x)

        return output

    def backward(self, d_values):
        # calculate the gradients with help of the derivative
        return sigmoid(d_values) * (1 - sigmoid(d_values))

### Implementation

Now let's put together the components you have implemented so far to our MLP:

In [140]:
class MLP_implementation():
    def __init__(self,
        input_size,
        output_size,
        hidden_layers,
        hidden_layers_size,
        hidden_activation_func,
        output_activation_function,
        loss_function,
    ):
        self.hidden_layers = hidden_layers
        self.hidden_layers_size = hidden_layers_size
        self.hidden_activation_func = hidden_activation_func
        self.loss_function = loss_function
        self.output_activation_function = output_activation_function
        self.layers = []

        # Initialize hidden layers
        for i in range(hidden_layers):
            if i == 0:
                layer = FeedForwardLayer(input_size, hidden_layers_size)
            else:
                layer = FeedForwardLayer(hidden_layers_size, hidden_layers_size)
            self.layers.append(layer)

        # Initialize output layer
        self.output_layer = FeedForwardLayer(hidden_layers_size, output_size)

    def forward_pass(self, x):
        for layer in self.layers:
            x = layer.forward(x)
            x = self.hidden_activation_func.forward(x)
        
        output = self.output_layer.forward(x)
        output = self.output_activation_function.forward(output)

        return output

    def backward_pass(self, d_values, learning_rate):
        # Backpropagate through output layer
        d_values = self.output_activation_function.backward(d_values)
        self.output_layer.backward(d_values, learning_rate)

        # Backpropagate through hidden layers
        for layer in reversed(self.layers):
            d_values = layer.backward(d_values, learning_rate)

    def train(self, input_data, targets, learning_rate=1, epochs=1):
        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            total_loss = 0.0
            random_order = np.random.permutation(len(input_data))
            for i in random_order:
                # Forward pass
                output = self.forward_pass(input_data[i])
                
                # Calculate loss
                loss = self.loss_function.forward(output, targets[i])
                total_loss += loss

                # Backward pass
                self.backward_pass(self.loss_function.backward(output, targets[i]), learning_rate)

            print(f"Average Loss: {total_loss / len(input_data)}")

    def inference(self, input_data):
        output = []
        for data in input_data:
            output.append(self.forward_pass(data))
            
        return np.array(output)


### MLP Inititialization

In [141]:
# Initialize MLP
xor_mlp = MLP_implementation(
    input_size=2,
    output_size=1,
    hidden_layers=3,
    hidden_layers_size=3,
    hidden_activation_func=ReluActivationFunction(),
    output_activation_function=SigmoidActivationFunction(),
    loss_function=BinaryCrossEntropy(),
)



### Training

In [142]:
# TODO
input_data = [[0,0],[1,0],[0,1],[1,1]]
targets = [0,1,1,0]

xor_mlp.train(input_data, targets, learning_rate=0.05, epochs=2500)

Epoch 1/2500
Input of the network, (2,)
Weights: (2, 3)
Biases: (3,)
[0 1]
[[-0.50099212 -0.82075324 -0.46263879]
 [-1.00010372  0.13515638 -0.8487896 ]]
[0. 0. 0.]
Output of the network, [-1.00010372  0.13515638 -0.8487896 ]


AttributeError: 'FeedForwardLayer' object has no attribute 'output_size'

### Evaluation

In [ ]:
# Test and evaluate your new model as in the previous task
# TODO
predictions = xor_mlp.inference(input_data)
print(predictions)
accuracy_value = accuracy(predictions, targets) 
print(accuracy_value)

[]


AssertionError: 

You will now be able to get to 100% accuracy on the XOR task with MLP!!

If you are interested, you can see this [demo](https://lecture-demo.iar.kit.edu/neural-network-demo/) to see how the decision boundaries are found by the MLPs.

## Excercise 3: XOR Task with MLP (using Pytorch)

Everything could have been much easier!

The excercises so far is only for you to undertand the internal details of training a neural network. In practice, we do not have to implement the forward and backward pass of the common function by hand. All can be taken care of by Pytorch!

Look up the Pytorch documentation, and fill in the following blocks of code to build the same MLP with Pytorch:

### Defining the model

In [ ]:
import torch
import torch.nn as nn

xor_mlp_pytorch = nn.Sequential(
    ...
)

### Initializing weights

In [ ]:
# Init weights
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

# Apply the initialization to the model
xor_mlp_pytorch.apply(init_weights)

### Loss Function: Binary Cross Entropy

In [ ]:
loss_fn = ...

### Optimizer: Stochastic gradient descent

In [ ]:
optimizer = ...

### Training

Below we provide you with a simple training loop.

For the first two epochs, print out the gradient and the values some weights of the network.

**Question**:
- Explain what happens after each step in the training loop.
- Why do we need `optimizer.zero_grad()` here? When should we NOT use it?

**Answer**: ...


In [ ]:
# Define our data
input_data_tensor = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float)
targets_tensor = torch.unsqueeze(
    torch.tensor([0,1,1,0], dtype=torch.float), 1
)

# Training loop
epochs = 2500
for epoch in range(epochs):

    optimizer.zero_grad()
    if epoch < 2:
        print(...)

    output = xor_mlp_pytorch(input_data_tensor)
    if epoch < 2:
        print(...)


    loss = loss_fn(output, targets_tensor)
    if epoch < 2:
        print(...)


    loss.backward()
    if epoch < 2:
        print(...)

    optimizer.step()
    if epoch < 2:
        print(...)


Follow the loss in the backward direction, using its `.grad_fn` attribute too see the computation graph:

In [ ]:
print(...)
print(...)
print(...)


### Evaluation

In [ ]:
predictions = ...
accuracy_value = ...
print(accuracy_value)


## Excercise 4: Iris Dataset 🌷 task with MLP (using Pytorch)

Iris is a genus of hundreds of species of flowering plants with showy flowers. The Iris data set consists of 150 samples from three species of Iris which are hard to distinguish (Iris setosa, Iris virginica and Iris versicolor). There are four features from each sample: the length and the width of the sepals and petals, in centimeters. Based on these features, the goal is to predict which species of Iris the sample belongs to.


For this exercise, you need to enable GPUs for this notebook:

- Navigate to "**Edit**" → "**Notebook Settings**"
- Select GPU from the **Hardware Accelerator** drop-down
- You might need to rerun the notebook

Next, we'll check if we can connect to the GPU with PyTorch:

In [ ]:
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

###  Loading Dataset

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch

iris = load_iris()
X, y = iris.data, iris.target
num_classes = 3



Process the data.

**Question**: Is there anything we need to do with the default target? Why?

**Answer**: ...

In [ ]:
# Split the dataset into training and test dataset
# TODO
X_train, X_test, y_train, y_test = ...

# Process the data
# TODO
y_train_one_hot = ...
y_test_one_hot = ...



### Architecture

We will again use an MLP for this task (with Pytorch).

Intitialize a model with **4 hidden layers** and a **hidden layer size of 768**.

**Question**: Is there any else we should change when building the MLP to fit this task?

**Hint**: it is no longer a binary classification problem

**Answer:** ...

In [ ]:
# Defining the model
xor_mlp_pytorch = nn.Sequential(
    ...
)

# Apply the initialization to the model
xor_mlp_pytorch.apply(init_weights)

# Defining loss function: Cross Entropy Loss
loss_fn = ...

# Defining optimizer: Stochastic gradient descent
optimizer = ...


### Training

As you have learnt from the lecture, we can speed up the training process by **batching** and using **GPUs**. Modify the following code for batching and GPUs.

You can also run the code before and after you make changes to see the speed up gain from batching and using GPU.

**Hints**: You can make use of Pytorch's `DataLoader`

**Question**: Report the execution time with and without GPU and batching.

**Answer**: ...

In [ ]:
# Training loop
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    output = xor_mlp_pytorch(X_train)
    loss = loss_fn(output, y_train_one_hot)
    loss.backward()
    optimizer.step()




### Evaluation

Show the overall accuracy of our model on the test dataset. Use the existing `accuracy` function that you implemented earlier.

In [ ]:
# TODO
predictions = ...
accuracy_value = ...
print(accuracy_value)

Print the confusion matrix using `sklearn.metrics.confusion_matrix`.

In [ ]:
# TODO
...

**Question**: Now also look at the confusion matrix, what can you conclude from it?

**Answer**: ...